In [ ]:
import numpy as np
from pprint import pprint

import random, pickle 
import pandas as pd

from sklearn.datasets import make_regression
# from autosklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split

import autosklearn
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.classification import AutoSklearnClassifier

In [ ]:
df = pd.read_pickle("data_latest_average_non_zero_raw_secondary_task_repeated_insole.pkl")

# # if we decided to remove duplicates in participant samples
# duplicate_rows = df.duplicated(subset=['participant_number','left_or_right'], keep='first')
# df = df[~duplicate_rows]

In [ ]:
df.columns

In [ ]:
participant_list = df.participant_number.drop_duplicates().to_list()
random.Random(420).shuffle(participant_list)
print(participant_list)

participant_list_train, participant_list_test = train_test_split(participant_list, random_state=115,  train_size=0.85)

print(participant_list_train, participant_list_test)

In [ ]:
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3)

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

df_test = df[df['participant_number'].isin(participant_list_test)] #select test participants
df_test = df_test.sample(frac=1).reset_index(drop=True) #shuffle and reset_index
# df_test.reset_index(inplace=True,drop=True)
df_train = df[df['participant_number'].isin(participant_list_train)] #select train participants
df_train = df_train.sample(frac=1).reset_index(drop=True) #shuffle and reset_index
# df_train.reset_index(inplace=True,drop=True)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_test.columns.tolist()[1:17]

In [ ]:

for feature_num, feature_name in enumerate(df_test.columns.tolist()[1:17]):


    print("Current Feature Classifier:", feature_name)

    y_train = df_train[[feature_name]].values
    x_train = np.asarray(df_train.input_features.tolist())


    y_test = df_test[[feature_name]].values

    x_test = np.asarray(df_test.input_features.tolist())

    

    # y_train = y_train[:,feature_num]
    # y_test  = y_test[:,feature_num]

    print(x_train.shape, y_train.shape , x_test.shape, y_test.shape)

    automl = AutoSklearnClassifier(
        time_left_for_this_task=3600,
        per_run_time_limit=180,
        memory_limit = 500000,
        seed = 14141,
        # metric = autosklearn.metrics.balanced_accuracy,
        
        # resampling_strategy = 'cv'
        # resampling_strategy_arguments = {
        # "shuffle": True,        # Whether to shuffle before splitting data
        # # "folds": 3              # Used in 'cv' based resampling strategies
        # }
    )
    automl.fit(x_train, y_train)

    # save model
    with open('data_averaged_time5hours_perRun300_secondary_task_'+ feature_name +'model.pkl', 'wb') as f:
        pickle.dump(automl, f)

    print(automl.leaderboard())

    pprint(automl.show_models(), indent=4)


    predictions = automl.predict(x_test)
    print("Accuracy score:", accuracy_score(y_test, predictions))

    df_test['predictions_' + feature_name] = predictions
    df_test['y_test_' + feature_name] = y_test

df_test.to_pickle("data_averaged_time5hours_perRun300_secondary_task_single_classifiers.pkl")
df_test.to_csv("data_averaged_time5hours_perRun300_secondary_task_single_classifiers.csv", index=False)

In [ ]:

# for feature_num, feature_name in enumerate(df_test.columns.tolist()[6:17]):


#     print("Current Feature Classifier:", feature_name)

#     y_train = df_train[[feature_name]].values
#     x_train = np.asarray(df_train.input_features.tolist())


#     y_test = df_test[[feature_name]].values

#     x_test = np.asarray(df_test.input_features.tolist())

    

#     # y_train = y_train[:,feature_num]
#     # y_test  = y_test[:,feature_num]

#     print(x_train.shape, y_train.shape , x_test.shape, y_test.shape)

#     automl = AutoSklearnClassifier(
#         time_left_for_this_task=3600,
#         per_run_time_limit=180,
#         memory_limit = 500000,
#         seed = 14141,
#         # metric = autosklearn.metrics.balanced_accuracy,
        
#         # resampling_strategy = 'cv'
#         # resampling_strategy_arguments = {
#         # "shuffle": True,        # Whether to shuffle before splitting data
#         # # "folds": 3              # Used in 'cv' based resampling strategies
#         # }
#     )
#     automl.fit(x_train, y_train)

#     # save model
#     with open('data_averaged_time5hours_perRun300_secondary_task_'+ feature_name +'model.pkl', 'wb') as f:
#         pickle.dump(automl, f)

#     print(automl.leaderboard())

#     pprint(automl.show_models(), indent=4)


#     predictions = automl.predict(x_test)
#     print("Accuracy score:", accuracy_score(y_test, predictions))

#     df_test['predictions_' + feature_name] = predictions
#     df_test['y_test_' + feature_name] = y_test

# df_test.to_pickle("data_averaged_time5hours_perRun300_secondary_task_single_classifiers.pkl")
# df_test.to_csv("data_averaged_time5hours_perRun300_secondary_task_single_classifiers.csv", index=False)